In [1]:
library(rdydisstools)
loadpackages(c('mirt', 'doParallel', 'foreach', 'logging', 'glue'))

In [9]:
load(file='~/dissertation/datamodels/hexaco.RData')

# Seed list generated using random.org query: 
# https://www.random.org/integer-sets/?sets=1&num=5000&min=1&max=1000000000&seqnos=on&commas=on&sort=on&order=index&format=html&rnd=new
# Date: 21-Aug-2018
seeds <- read.table('~/dissertation/seeds/simulationSeeds.txt', header=FALSE, sep=',')$V1

nObs <- 500
nDatasets <- 5000
ls <- crModel(label='longstring', mu=.02, sigma=.01)
cent <- crModel(label='centered', mu=.06, sigma=.02)

registerDoParallel(24)

foreach(i=1:nDatasets, .verbose=TRUE) %dopar% {
    addHandler(writeToFile, file=glue::glue('~/dissertation/logs/simulation/sim{i}.log'), level='INFO')
    logstring <- glue::glue('dis.simulation.{i}')
    
    set.seed(seeds[i])
    logging::loginfo(glue::glue('seed has been set to: {seeds[i]}'), logger=logstring) 
    
    f <- simFactorScores(hexaco$factorCorrelations, nObs)
    logging::loginfo('Factor scores have been simulated successfully', logger=logstring)
    
    df <- mirt::simdata(hexaco$a, hexaco$d, itemtype='graded', Theta=f, mins=1)
    logging::loginfo('Item-level data has been simulated successfully', logger=logstring)
    
    colnames(df) <- rownames(hexaco$itemParameters)
    logging::loginfo('Column names of dataset have been set from hexaco itemParameters', logger=logstring)
    
    nLs <- getProportionFromModel(ls, nObs)
    nCent <- getProportionFromModel(cent, nObs)
    logging::loginfo(glue::glue('Dataset has been assigned {nLs} longstring respondents and {nCent} centered respondents'), logger=logstring)
    
    whichCL <- sample(nObs, sum(nLs, nCent))
    cl <- cbind(rep(0,nObs), rep(NA, nObs))
    colnames(cl) <- c('careless', 'crModel')
    logging::loginfo('Columns have been created with placeholders for careless designation and type', logger=logstring)

    
    cl[whichCL,1] <- 1
    cl[whichCL,2] <- recursiveAssign(whichCL, c(nLs, nCent), c('longstring','centered'))
    logging::loginfo('Designation and type has been populated for all respondents', logger=logstring)
    
    df <- cbind(as.data.frame(df), as.data.frame(cl))
    logging::loginfo('Dataset is ready for injection of careless respondents', logger=logstring)
    
    careless <- df[,'careless'] == 1
    df[careless,] <- t(apply(df[careless,], 1, simCareless, logstring=logstring, floor=1, ceiling=7))
    df[,1:101] <- apply(df[1:101], 2, as.numeric)
    logging::loginfo('Careless respondents have been injected into dataset')
    
    save(df, file=glue::glue('~/dissertation/data/simsample{i}.RData'))
    logging::loginfo('Simulation complete: dataset has been saved to datasets', logger = logstring)
}

numValues: 1, numResults: 0, stopped: TRUE
got results for task 1
numValues: 1, numResults: 1, stopped: TRUE
calling combine function
evaluating call object to combine results:
  fun(accum, result.1)
returning status TRUE


[[1]]
NULL